In [2]:
import sys
sys.path.append("../scripts/")
from cleaner import DataCleaner
from assistants import assistant
from text_preprocessor import Processor

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import warnings

from data_describe.text.text_preprocessing import *
from sklearn.datasets import fetch_20newsgroups

warnings.filterwarnings('ignore')

In [4]:
assistant = assistant()
cleaner = DataCleaner()
processor = Processor()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mikyg\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mikyg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mikyg\AppData\Roaming\nltk_data...
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mikyg\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mikyg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mikyg\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [6]:
repo="https://github.com/michaelgetachew-abebe/Prompt-Engineering-LLMs.git"
news_dataframe=assistant.read_from_dvc("data/cleaned_news.csv",repo,"60787a4c729bc276d60d03c0c0823a974cf1c93e",low_memory=False)
news_dataframe=news_dataframe.sample(frac=1)

In [7]:
train_news=news_dataframe.head(7)
test_news=news_dataframe.tail(3)

In [8]:
full_processed_df=processor.prepare_text(train_news.copy())
description_processed=processor.prepare_text(train_news.copy(),columns=["Description"])[["Description","Analyst_Average_Score"]]
title_processed=processor.prepare_text(train_news.copy(),columns=["Title"])[["Title","Analyst_Average_Score"]]
body_processed=processor.prepare_text(train_news.copy(),columns=["Body"])[["Body","Analyst_Average_Score"]]
unprocessed=train_news

processed_test=processor.prepare_text(test_news)

In [9]:
frames = [full_processed_df, description_processed, title_processed, body_processed, unprocessed]

full_promp_trainer = pd.concat(frames)

In [10]:
full_promp_trainer=full_promp_trainer.reset_index().drop(['index'],axis=1)

In [11]:
full_promp_trainer=full_promp_trainer[['Domain','Title','Description','Body','timestamp','Analyst_Average_Score']]
full_promp_trainer.rename(columns={"Analyst_Average_Score":"Analyst Average Score"},inplace=True)
# full_promp_trainer=cleaner.convert_to_datetime(full_promp_trainer,['timestamp'])
full_promp_trainer=full_promp_trainer.fillna('')
full_promp_trainer.head()

,Domain,Title,Description,Body,timestamp,Analyst Average Score
0,eminetra za,7th anniversari scoan collaps nigeria sabc news,lago nigeria 84 south african kill build hous ...,today 7th anniversari tragic collaps synagogu ...,2021 09 12 05 17 50 279081,0.0
1,rassegnastampa news,bori johnson use taxpay fund jet elect campaig...,trigger protest vote upset minist breach minis...,bori johnson use taxpay fund jet elect campaig...,2021 09 09 18 17 46 258006,0.0
2,kryptogazett com,triacetin vertrieb markt 2021 global unternehm...,abschnitten und endanwendungen organisationen ...,global triacetin vertrieb markt 2021 von herst...,2021 09 08 12 47 46 078369,0.0
3,twitter com,stumbl case woman face evict home inherit brot...,stumbl case woman face evict home inherit brot...,stumbl case woman face evict home inherit brot...,2021 09 08 13 02 45 802298,0.0
4,manometcurr com,global region beta caroten market research 202...,key player dsm basf alli biotech chr histor ma...,global region beta caroten market research 202...,2021 09 13 03 02 45 609228,0.0


In [12]:
processor.prepare_tuner(full_promp_trainer)

tuner prepared successfuly


In [13]:
full_processed_df.to_csv("../data/training_news.csv",index=False)
processed_test.to_csv("../data/testing_news.csv",index=False)